In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import matplotlib as mpl
import pypsa
import glob
import yaml

from pypsa.networkclustering import get_clustering_from_busmap
from pypsa.descriptors import get_switchable_as_dense as get_as_dense
from matplotlib.animation import FuncAnimation

In [16]:
with open('../config.yaml', 'r') as configfile:
    config = yaml.safe_load(configfile)

In [17]:
### config to be put in config.yaml

TY = config['scenario']['target_years'][0]

simulation_year = config['scenario']['simulation_years'][0]

scenario = f'National estimates {TY}'

network_f = f'../networks/pilot_elec-vre_TY{TY}_{simulation_year}.nc'

In [18]:
def aggregate_links(network, remove_zero_capacity_links=False):
    links_list = network.links.index
    p_max_pu = get_as_dense(network, 'Link', 'p_max_pu', network.snapshots)
    p_min_pu = get_as_dense(network, 'Link', 'p_min_pu', network.snapshots)
    new_p_min_pu = pd.DataFrame()
    new_p_max_pu = pd.DataFrame()
    new_p_nom = pd.Series()
    droplinks = []
    for link in links_list:

        if link in droplinks: continue
        
        bus0, bus1 = network.links.loc[link, ['bus0', 'bus1']]
        forward_p_max_pu = p_max_pu[link]
        bidirectional = p_min_pu[link].min() < 0.
        if not bidirectional:
            try:
                reverse_direction_link = network.links.index[(network.links.bus0==bus1) & (network.links.bus1==bus0)][0]
            except:
                print(f'Warning: Link {link} is not bidirectional but seems not to have a reverse direction link neither...')
                continue
                
            reverse_p_max_pu = p_max_pu[reverse_direction_link]
            p_nom = np.max([network.links.p_nom[link], network.links.p_nom[reverse_direction_link]])

            new_p_max_pu[link] = forward_p_max_pu.mul(network.links.p_nom[link]) / p_nom
            new_p_min_pu[link] = -reverse_p_max_pu.mul(network.links.p_nom[reverse_direction_link]) / p_nom
            new_p_nom[link] = p_nom
            droplinks.append(reverse_direction_link)

            if (remove_zero_capacity_links) & (p_nom == 0.):
                print('yes')
                droplinks.append(link)
                new_p_min_pu.drop(link, inplace=True)
                new_p_max_pu.drop(link, inplace=True)

    return((droplinks, new_p_nom, new_p_max_pu, new_p_min_pu))

In [19]:
network = pypsa.Network(network_f)
network.add('Line', name='dummy', bus0='DE00', bus1='NL00', s_nom=0) ### get_clustering_from_busmap throws an error without lines ... remove line after clustering

INFO:pypsa.io:Imported network pilot_elec-vre_TY2030_2000.nc has buses, carriers, generators, links, loads, storage_units


In [20]:
### aggregate links
droplinks, new_p_nom, new_p_max_pu, new_p_min_pu = aggregate_links(network, remove_zero_capacity_links=True)

network.mremove('Link', droplinks)
network.links_t.p_max_pu.loc[:,new_p_max_pu.columns] = new_p_max_pu
network.links_t.p_min_pu.loc[:,new_p_min_pu.columns] = new_p_min_pu

In [21]:
busmap = {bus: bus[:2] for bus in network.buses.index}
Clustering = get_clustering_from_busmap(network, busmap, with_time=True, aggregate_one_ports={'Generator', 'Load'})

       'ITCN-ITCS HVDC', 'ITCN-ITN1 HVAC', 'ITCN-ITSA HVDC', 'ITCS-ITCN HVDC',
       'ITCS-ITS1 HVAC', 'ITCS-ITSA HVDC', 'ITCS-ITVI HVDC', 'ITS1-ITCA HVAC',
       'ITSA-ITVI HVDC', 'ITVI-ITSI HVDC', 'NOM1-NON1 HVAC', 'SE02-SE03 HVAC',
       'UKNI-UK00 HVDC'],
      dtype='object', name='Link') for attribute p_min_pu of Link are not in main components dataframe links
       'ITCN-ITCS HVDC', 'ITCN-ITN1 HVAC', 'ITCN-ITSA HVDC', 'ITCS-ITCN HVDC',
       'ITCS-ITS1 HVAC', 'ITCS-ITSA HVDC', 'ITCS-ITVI HVDC', 'ITS1-ITCA HVAC',
       'ITSA-ITVI HVDC', 'ITVI-ITSI HVDC', 'UKNI-UK00 HVDC'],
      dtype='object', name='Link') for attribute p_max_pu of Link are not in main components dataframe links


In [22]:
n_clustered = Clustering.network
n_clustered.name = 'Simple DestinE Pilot with VRE profiles and hydro'
#n_clustered.consistency_check()
n_clustered.mremove('Line', n_clustered.lines.index)

In [23]:
n_clustered.links_t.p_max_pu.drop(n_clustered.links_t.p_max_pu.columns[~n_clustered.links_t.p_max_pu.columns.isin(n_clustered.links.index)], axis=1, inplace=True)
n_clustered.links_t.p_min_pu.drop(n_clustered.links_t.p_min_pu.columns[~n_clustered.links_t.p_min_pu.columns.isin(n_clustered.links.index)], axis=1, inplace=True)

In [24]:
#n_clustered.export_to_netcdf(f'../networks/pilot_elec-vre-hydro_TY{TY}_{simulation_year}_simpl.nc')